# Bloom Filters (APS project)

### This project contains two applications of Bloom Filters namely 
* searching in multiple databases / caches
* username / phone number validation

## Section 1 : basic Bloom Filter

In [1]:
import math 
import mmh3 
from bitarray import bitarray

In [2]:
class BloomFilter(object):
    def __init__(self, items_count,fp_prob): 
        self.fp_prob = fp_prob 
        self.size = self.get_size(items_count,fp_prob) 
        self.hash_count = self.get_hash_count(self.size,items_count) 
        self.bit_array = bitarray(self.size) 
        self.bit_array.setall(0) 

    def add(self, item):
        digests = [] 
        for i in range(self.hash_count): 
            digest = mmh3.hash(item,i) % self.size 
            digests.append(digest)
            self.bit_array[digest] = True

    def check(self, item): 
        for i in range(self.hash_count): 
            digest = mmh3.hash(item,i) % self.size 
            if self.bit_array[digest] == False:
                return False
        return True

    @classmethod
    def get_size(self,n,p):
        m = -(n * math.log(p))/(math.log(2)**2) 
        return int(m) 

    @classmethod
    def get_hash_count(self, m, n):
        k = (m/n) * math.log(2) 
        return int(k)

In [3]:
from random import shuffle


In [4]:
with open('words_alpha.txt') as word_file:
    valid_words = set(word_file.read().split())

In [5]:
len(valid_words)

370099

In [6]:
n = 466544
p = 0.05

In [7]:
bloom = BloomFilter(n,p)

In [8]:
print("Size of bit array:{}".format(bloom.size)) 
print("False positive Probability:{}".format(bloom.fp_prob)) 
print("Number of hash functions:{}".format(bloom.hash_count)) 

Size of bit array:2909006
False positive Probability:0.05
Number of hash functions:4


In [9]:
for item in valid_words: 
    bloom.add(item)

In [10]:
bloom.check('abasic')

True

In [11]:
bloom.check('dhfgskfgs')

False

In [12]:
bloom.check('apple')

True

In [13]:
def basic_check(value):
    for word in valid_words:
        if word == value:
            print(True)
            break

In [14]:
import time
start_time = time.time()
basic_check('apple')
print("Time for Normal search --- %s seconds ---" % (time.time() - start_time))
nst = time.time()
bloom.check('apple')
print("Time for bloom filters lookup --- %s seconds ---" % (time.time() - nst))
asd = time.time()
for word in valid_words:
        if word == 'apple':
            print(True)
            break
print("Time for word lookup --- %s seconds ---" % (time.time() - asd))

True
Time for Normal search --- 0.07300186157226562 seconds ---
Time for bloom filters lookup --- 7.677078247070312e-05 seconds ---
True
Time for word lookup --- 0.058219194412231445 seconds ---


## Section 2 : Applications

## Application 1 : searching in multiple databases / caches

In [15]:
lines_per_file = 100000
smallfile = None
with open('words_alpha.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        if lineno % lines_per_file == 0:
            if smallfile:
                smallfile.close()
            small_filename = 'caches/database_{}.txt'.format(lineno + lines_per_file)
            smallfile = open(small_filename, "w")
        smallfile.write(line)
    if smallfile:
        smallfile.close()

In [16]:
# create multiples bloom filters for every database
n = 100000
p = 0.05

In [17]:
import os
multiple_bloom = []
path = 'caches/'

In [18]:
for filename in os.listdir(path):
    filename = path+filename
    bloom = BloomFilter(n,p)
    with open(filename) as word_file:
        valid_words = set(word_file.read().split())
    for item in valid_words: 
        bloom.add(item)
    multiple_bloom.append(bloom)

In [ ]:
def mul_check(value):
    for i in range(1,5):
        if multiple_bloom[i-1].check(value):
            print(i)

In [ ]:
mul_check('potato')

2
4


In [ ]:
mul_check('zebra')

2


In [ ]:
def trad_check(value):
    for filename in os.listdir(path):
        filename = path+filename
        with open(filename) as word_file:
            words = set(word_file.read().split())
            if value in words:
                print(filename)

In [ ]:
trad_check('potato')

caches/database_300000.txt


In [ ]:
trad_check('zebra')

caches/database_400000.txt


In [ ]:
# comparision in speed

In [ ]:
start_time = time.time()
trad_check('potato')
print("Time for Normal search --- %s seconds ---" % (time.time() - start_time))


nst = time.time()
mul_check('apple')
print("Time for bloom filters lookup --- %s seconds ---" % (time.time() - nst))

caches/database_300000.txt
Time for Normal search --- 0.4532809257507324 seconds ---
1
2
Time for bloom filters lookup --- 0.0001342296600341797 seconds ---


## Application 2 : new username / phone number validation

In [ ]:
from random import randint

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [ ]:
with open('caches/phone_numbers.txt','w') as ff:
    for i in range(1000000):
        ff.write(str(random_with_N_digits(10))+'\n')

In [ ]:
n = 1000000
p = 0.05
num_bloom = BloomFilter(n,p)

In [ ]:
with open('caches/phone_numbers.txt') as number_file:
        used_nums = set(number_file.read().split())
for item in used_nums: 
    num_bloom.add(item)

In [ ]:
num_bloom.check('9371134560')

In [ ]:
num_bloom.check('4852465447')

In [ ]:
def basic_num_check(num):
    for number in used_nums:
        if number == num:
            print(True)

In [ ]:
basic_num_check('9371134560')

In [ ]:
start_time = time.time()
num_bloom.check('9371134560')
blommtt = time.time() - start_time
print("Time for Normal search --- %s seconds ---" % (blommtt))


nst = time.time()
basic_num_check('9371134560')
nstt = time.time() - nst
print("Time for bloom filters lookup --- %s seconds ---" % (nstt))

In [ ]:
print(nstt - blommtt)